In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [33]:
df = pd.read_csv("../input/growthdata92_02.csv",index_col=0)
x = df.drop(['iso3','ln_y','growth'], axis=1)
y1 = df['ln_y']
y2 = df['growth']

df_0211 = pd.read_csv("../input/growthdata02_11.csv",index_col=0)
x_0211 = df_0211.drop(['iso3','ln_y','growth'], axis=1)
y1_0211 = df_0211['ln_y']
y2_0211 = df_0211['growth']

x1_train,x1_test,y1_train,y1_test = train_test_split(x,y1,test_size = 0.2)
x2_train,x2_test,y2_train,y2_test = train_test_split(x,y2,test_size = 0.2)

In [39]:
alphas = [i for i in range(1, 101)]
for x_train,x_test, y, y_train, y_test,y_0211 in [(x1_train,x1_test,y1, y1_train, y1_test, y1_0211),
                                                  (x2_train,x2_test,y2, y2_train, y2_test, y2_0211)]:
    rf_scores = []
    for alpha in alphas:
        rf = RandomForestRegressor(n_estimators = alpha)
        scores = cross_val_score(rf, x_train, y_train, cv=10)
        rf_scores.append(np.mean(scores))
    best_alpha_index = np.argmax(rf_scores)  
    best_alpha = alphas[best_alpha_index]  
    best_score = rf_scores[best_alpha_index] 
    print('Best score: ' + str(best_score))
    print('Best alpha: ' + str(best_alpha))
    # Ex1
    rf = RandomForestRegressor(n_estimators = best_alpha)
    rf.fit(x_train, y_train)
    mean = pd.DataFrame(np.mean(x_train)).transpose()
    naive_y_pre = rf.predict(mean)
    mae = abs(naive_y_pre[0] - np.mean(y_train))
    print('MAE for naive predictor: ' + str(mae))
    sink_y_pre = rf.predict(x)
    rmse = np.sqrt(mean_squared_error(y, sink_y_pre))
    print('RMSE for kitchen sink predictor: '+ str(rmse))
    test_y_pre = rf.predict(x_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, test_y_pre))
    print('Performance on test set: ' + str(test_rmse))
    # Ex2
    test_y_0211_pre = rf.predict(x_0211)
    test_rmse_0211 = np.sqrt(mean_squared_error(y_0211, test_y_0211_pre))
    print('Performance on 02-11 dataset: ' + str(test_rmse_0211))


Best score: 0.8513559040740208
Best alpha: 94
MAE for naive predictor: 0.06768653325525972
RMSE for kitchen sink predictor: 0.22983315505328836
Performance on test set: 0.4296410499790884
Performance on 02-11 dataset: 0.3703576196369367
Best score: 0.016529065776454778
Best alpha: 46
MAE for naive predictor: 0.0008251380612412165
RMSE for kitchen sink predictor: 0.016276197492782554
Performance on test set: 0.03041289116127451
Performance on 02-11 dataset: 0.02981989645251401
